In [9]:
import cv2
import matplotlib.pyplot as plt
import imutils
from itertools import combinations
import math
import numpy as np
import datetime
import os
import tensorflow as tf

In [10]:
model = tf.keras.models.load_model("model.h5")

In [11]:
#Load model files and graph
config_file='MobileNetSSD_deploy.prototxt'
frozen_model = 'MobileNetSSD_deploy.caffemodel'
#Create model object
p_model = cv2.dnn_DetectionModel(frozen_model, config_file)

In [14]:
#Specify the Labels
classLabels = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]


In [42]:
cap = cv2.VideoCapture()

labels= ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

font_scale = 2
font = cv2.FONT_HERSHEY_PLAIN

def get_index(p):
    p = p.flatten()
    p = p.tolist()
    max = 0
    ind = 0
    for i in range(len(p)):
        if max < p[i]:
            max = p[i]
            print('Max: ', max)
            ind = i
    return ind
        

#set input frame properties
p_model.setInputSize(320,320)
p_model.setInputScale(1.0/127.5)
p_model.setInputMean((127.5,127.5,127.5))
p_model.setInputSwapRB(True)

#Check if the video is opened correctly
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot Open video")
    
while True:
    ret, frame = cap.read()
    if ret == False:
        cap.release()
        cv.destroyAllWindows()
        break
    ClassIndex, confidence, bbox = p_model.detect(frame, confThreshold = 0.55)
    
     #If no object is detected
    if(len(ClassIndex) == 0):
        cv2.imshow('Video feed', frame)
        
    if(len(ClassIndex)  ==1):
        cv2.rectangle(frame, bbox, (0,255,0), 2)
    
    indices=[]
    
    #If some objects are detected, remove the object if it does not falls within one of the required Labels
    if(len(ClassIndex) != 0):
        ClassIndex = ClassIndex.flatten()
        print(ClassIndex, ClassIndex.shape)
        for i in range(ClassIndex.shape[0]):
            if ClassIndex[i]>80:
                indices.append(i)
                continue
            if classLabels[ClassIndex[i]] not in ["person"]:
                print(classLabels[ClassIndex[i]])
                indices.append(i)
        ClassIndex1 = np.delete(ClassIndex, indices)
        print(ClassIndex1)
        mapped = zip(ClassIndex1.flatten(), confidence.flatten(), bbox)
        
        
        for ob in mapped:
            cv2.rectangle(frame, (ob[2][0], ob[2][1], ob[2][2], ob[2][3]), (0,255,0), 2)
            print(ob[2])
            crop_img = frame[ob[2][1]: ob[2][3], ob[2][0]: ob[2][2]]
            print('Crop Image: ',crop_img.shape)
            if crop_img.shape[0] > 100:
                
                crop_img = cv2.resize(crop_img,(48,48),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
                crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
                print(crop_img.shape)
                crop_img = crop_img.reshape(1,48,48,1)
                prediction = model.predict(crop_img)
                print('prediction: ', prediction)
                ind = get_index(prediction)
                print('index : ', ind)
            cv2.putText(frame, labels[ind], (ob[2][0]+10, ob[2][1]+40), font, fontScale= font_scale, color=(0,255,0), thickness=3) 
            
    cv2.imshow('Video feed', frame)                     
    
    if cv2.waitKey(2) & 0xFF == ord('q'): #close the program on pressing 'q'
        break
        
cap.release()#release input video
cv2.destroyAllWindows()
    

[15] (1,)
[15]
[201 134 432 337]
Crop Image:  (203, 231, 3)
(48, 48)
prediction:  [[2.22191826e-01 4.59094408e-05 3.18135023e-02 9.80066359e-02
  5.29547036e-01 7.40525499e-03 1.10989876e-01]]
Max:  0.22219182550907135
Max:  0.5295470356941223
index :  4
[15] (1,)
[15]
[203 135 428 336]
Crop Image:  (201, 225, 3)
(48, 48)
prediction:  [[1.3830145e-01 1.2009502e-04 1.9711671e-02 1.9063139e-01 4.4111615e-01
  7.7413786e-03 2.0237790e-01]]
Max:  0.13830144703388214
Max:  0.19063138961791992
Max:  0.4411161541938782
index :  4
[15] (1,)
[15]
[204 134 428 337]
Crop Image:  (203, 224, 3)
(48, 48)
prediction:  [[0.12161689 0.00053142 0.02732553 0.16231988 0.48296297 0.00964123
  0.19560203]]
Max:  0.12161688506603241
Max:  0.16231988370418549
Max:  0.482962965965271
index :  4
[15] (1,)
[15]
[205 133 427 337]
Crop Image:  (204, 222, 3)
(48, 48)
prediction:  [[1.00191556e-01 4.16411873e-04 1.99943054e-02 1.74087361e-01
  5.91585875e-01 5.11302752e-03 1.08611450e-01]]
Max:  0.10019155591726303


[15] (1,)
[15]
[208 135 424 336]
Crop Image:  (201, 216, 3)
(48, 48)
prediction:  [[0.14596224 0.00093167 0.02056241 0.24754514 0.40005192 0.01903409
  0.16591252]]
Max:  0.14596223831176758
Max:  0.24754513800144196
Max:  0.40005192160606384
index :  4
[15] (1,)
[15]
[207 137 423 334]
Crop Image:  (197, 216, 3)
(48, 48)
prediction:  [[0.13106678 0.00096844 0.01421673 0.32811522 0.36751252 0.02439768
  0.13372265]]
Max:  0.13106678426265717
Max:  0.32811522483825684
Max:  0.3675125241279602
index :  4
[15] (1,)
[15]
[206 135 425 336]
Crop Image:  (201, 219, 3)
(48, 48)
prediction:  [[8.5247487e-02 1.8246668e-04 1.5211802e-02 3.4492433e-01 4.0706605e-01
  1.5159265e-02 1.3220868e-01]]
Max:  0.08524748682975769
Max:  0.34492433071136475
Max:  0.4070660471916199
index :  4
[15] (1,)
[15]
[204 136 426 335]
Crop Image:  (199, 222, 3)
(48, 48)
prediction:  [[1.5250565e-01 2.6954574e-04 2.3138782e-02 2.2478732e-01 4.8660055e-01
  2.5167713e-02 8.7530479e-02]]
Max:  0.15250565111637115
Max:  0

prediction:  [[2.5476956e-01 2.2070695e-04 1.9730471e-01 6.4607754e-02 4.4785792e-01
  8.3326595e-03 2.6906746e-02]]
Max:  0.25476956367492676
Max:  0.44785791635513306
index :  4
[15] (1,)
[15]
[168  93 453 372]
Crop Image:  (279, 285, 3)
(48, 48)
prediction:  [[3.1509584e-01 3.4343571e-04 2.9963640e-02 4.7922201e-02 5.6336725e-01
  6.4639971e-03 3.6843609e-02]]
Max:  0.31509584188461304
Max:  0.5633672475814819
index :  4
[15] (1,)
[15]
[171  95 451 370]
Crop Image:  (275, 280, 3)
(48, 48)
prediction:  [[3.1854773e-01 6.8630681e-05 1.3330479e-01 5.2035455e-02 3.9434260e-01
  1.1911036e-02 8.9789696e-02]]
Max:  0.31854772567749023
Max:  0.3943426012992859
index :  4
[15] (1,)
[15]
[182  94 442 371]
Crop Image:  (277, 260, 3)
(48, 48)
prediction:  [[8.6557835e-02 9.3462717e-05 3.9604396e-02 1.2510294e-01 7.1854091e-01
  2.3597151e-03 2.7740685e-02]]
Max:  0.08655783534049988
Max:  0.1251029372215271
Max:  0.7185409069061279
index :  4
[15] (1,)
[15]
[182  95 442 370]
Crop Image:  (275,

prediction:  [[1.3296918e-02 1.5277941e-04 5.1449813e-02 4.5788655e-01 3.6260846e-01
  8.3897226e-03 1.0621578e-01]]
Max:  0.013296918012201786
Max:  0.051449812948703766
Max:  0.45788654685020447
index :  3
[15] (1,)
[15]
[200 106 426 360]
Crop Image:  (254, 226, 3)
(48, 48)
prediction:  [[0.01775699 0.00233754 0.13279854 0.38031363 0.2706592  0.03591474
  0.16021942]]
Max:  0.0177569929510355
Max:  0.13279853761196136
Max:  0.3803136348724365
index :  3
[15] (1,)
[15]
[202 107 423 359]
Crop Image:  (252, 221, 3)
(48, 48)
prediction:  [[0.02316457 0.00165908 0.06559096 0.45225447 0.22830473 0.02062969
  0.20839646]]
Max:  0.023164570331573486
Max:  0.06559096276760101
Max:  0.4522544741630554
index :  3
[15] (1,)
[15]
[200 110 426 356]
Crop Image:  (246, 226, 3)
(48, 48)
prediction:  [[1.5822852e-02 1.5851996e-04 5.3662509e-02 3.0406776e-01 5.5162609e-01
  1.2547242e-02 6.2114984e-02]]
Max:  0.01582285203039646
Max:  0.053662508726119995
Max:  0.3040677607059479
Max:  0.55162608623504

[15] (1,)
[15]
[201 130 431 341]
Crop Image:  (211, 230, 3)
(48, 48)
prediction:  [[0.21269514 0.00051879 0.0698601  0.25522837 0.41337585 0.00375704
  0.04456472]]
Max:  0.2126951366662979
Max:  0.2552283704280853
Max:  0.4133758544921875
index :  4
[15 15] (2,)
[15 15]
[203 129 431 341]
Crop Image:  (212, 228, 3)
(48, 48)
prediction:  [[0.24188311 0.00220166 0.06332035 0.15185547 0.48511815 0.00506592
  0.05055533]]
Max:  0.24188311398029327
Max:  0.48511815071105957
index :  4
[  1 295 113 185]
Crop Image:  (0, 112, 3)
[15] (1,)
[15]
[201 126 434 346]
Crop Image:  (220, 233, 3)
(48, 48)
prediction:  [[0.2931175  0.00084551 0.01909486 0.35262567 0.29979813 0.00172918
  0.03278912]]
Max:  0.293117493391037
Max:  0.35262566804885864
index :  3
[15] (1,)
[15]
[201 128 433 343]
Crop Image:  (215, 232, 3)
(48, 48)
prediction:  [[0.17999604 0.00123331 0.04467844 0.38799518 0.3342779  0.00296347
  0.04885562]]
Max:  0.1799960434436798
Max:  0.387995183467865
index :  3
[15] (1,)
[15]
[203 1

...

prediction:  [[3.3401000e-01 2.4275132e-04 1.1626816e-01 2.7718297e-01 1.3833785e-01
  5.5690754e-02 7.8267477e-02]]
Max:  0.3340100049972534
index :  0
[15] (1,)
[15]
[180 145 438 325]
Crop Image:  (180, 258, 3)
(48, 48)
prediction:  [[0.24212672 0.00048073 0.09680262 0.30641082 0.24224469 0.05358789
  0.05834653]]
Max:  0.2421267181634903
Max:  0.3064108192920685
index :  3
[15] (1,)
[15]
[179 147 442 323]
Crop Image:  (176, 263, 3)
(48, 48)
prediction:  [[6.1871789e-02 1.5934992e-04 9.6119203e-02 3.2572213e-01 2.3036687e-01
  1.2322240e-01 1.6253829e-01]]
Max:  0.061871789395809174
Max:  0.09611920267343521
Max:  0.3257221281528473
index :  3
[15] (1,)
[15]
[179 145 442 325]
Crop Image:  (180, 263, 3)
(48, 48)
prediction:  [[4.9005959e-02 6.2479849e-05 4.7952194e-02 4.6757779e-01 1.9692954e-01
  8.4932245e-02 1.5353978e-01]]
Max:  0.049005959182977676
Max:  0.4675777852535248
index :  3
[15] (1,)
[15]
[182 147 431 323]
Crop Image:  (176, 249, 3)
(48, 48)
prediction:  [[0.20053278 0.

prediction:  [[6.2443918e-01 3.2252360e-06 2.8584677e-01 3.6815491e-02 2.4595216e-02
  2.5788479e-02 2.5116359e-03]]
Max:  0.6244391798973083
index :  0
[15] (1,)
[15]
[171 162 451 309]
Crop Image:  (147, 280, 3)
(48, 48)
prediction:  [[1.6202423e-01 3.8301170e-07 6.5586829e-01 9.4790600e-02 3.9249614e-02
  3.1608991e-02 1.6457817e-02]]
Max:  0.16202422976493835
Max:  0.6558682918548584
index :  2
[15] (1,)
[15]
[169 164 442 309]
Crop Image:  (145, 273, 3)
(48, 48)
prediction:  [[7.26798102e-02 1.97967256e-06 1.42769888e-01 4.00004268e-01
  1.70103893e-01 1.02068439e-01 1.12371705e-01]]
Max:  0.07267981022596359
Max:  0.14276988804340363
Max:  0.4000042676925659
index :  3
[15] (1,)
[15]
[168 164 454 309]
Crop Image:  (145, 286, 3)
(48, 48)
prediction:  [[3.6700445e-01 1.3395227e-08 1.6694959e-01 1.5920103e-01 9.2548542e-02
  1.4739854e-02 1.9955660e-01]]
Max:  0.3670044541358948
index :  0
[15] (1,)
[15]
[167 173 411 307]
Crop Image:  (134, 244, 3)
(48, 48)
prediction:  [[5.1346060e-0

prediction:  [[2.0996344e-01 4.2120122e-05 6.6252895e-02 2.3399754e-01 1.7424409e-01
  4.4887919e-02 2.7061200e-01]]
Max:  0.20996344089508057
Max:  0.23399753868579865
Max:  0.2706120014190674
index :  6
[15] (1,)
[15]
[166 172 459 301]
Crop Image:  (129, 293, 3)
(48, 48)
prediction:  [[3.4034422e-01 1.5282448e-07 3.6665797e-01 3.3371601e-02 3.5466522e-02
  9.7354330e-02 1.2680519e-01]]
Max:  0.34034422039985657
Max:  0.36665797233581543
index :  2
[15] (1,)
[15]
[165 168 457 305]
Crop Image:  (137, 292, 3)
(48, 48)
prediction:  [[2.7403674e-01 7.9831725e-06 1.2926508e-01 1.4130938e-01 9.0456992e-02
  4.5620937e-02 3.1930289e-01]]
Max:  0.2740367352962494
Max:  0.31930288672447205
index :  6
[15] (1,)
[15]
[164 172 419 308]
Crop Image:  (136, 255, 3)
(48, 48)
prediction:  [[0.34000546 0.00055717 0.07637129 0.47539818 0.083079   0.00438921
  0.02019964]]
Max:  0.34000545740127563
Max:  0.4753981828689575
index :  3
[15] (1,)
[15]
[166 172 449 301]
Crop Image:  (129, 283, 3)
(48, 48)
pr

prediction:  [[0.1269353  0.00063792 0.14190553 0.49032536 0.11182564 0.03453402
  0.09383614]]
Max:  0.12693530321121216
Max:  0.1419055312871933
Max:  0.49032536149024963
index :  3
[15] (1,)
[15]
[164 167 458 306]
Crop Image:  (139, 294, 3)
(48, 48)
prediction:  [[6.8451956e-02 1.3403518e-05 1.1176940e-01 2.0908782e-01 1.4635164e-01
  5.3790640e-02 4.1053513e-01]]
Max:  0.06845195591449738
Max:  0.11176940053701401
Max:  0.2090878188610077
Max:  0.41053512692451477
index :  6
[15] (1,)
[15]
[165 170 455 303]
Crop Image:  (133, 290, 3)
(48, 48)
prediction:  [[7.5196542e-02 1.7651208e-05 1.3484155e-01 2.6418170e-01 1.2759915e-01
  5.2057859e-02 3.4610546e-01]]
Max:  0.07519654184579849
Max:  0.13484154641628265
Max:  0.2641817033290863
Max:  0.3461054563522339
index :  6
[15] (1,)
[15]
[165 168 451 305]
Crop Image:  (137, 286, 3)
(48, 48)
prediction:  [[2.6204211e-01 8.6042448e-05 6.8474673e-02 1.5207148e-01 7.1426302e-02
  1.6440226e-01 2.8149709e-01]]
Max:  0.2620421051979065
Max:  

[15] (1,)
[15]
[161 172 424 308]
Crop Image:  (136, 263, 3)
(48, 48)
prediction:  [[7.5979628e-02 1.4029515e-04 3.8270764e-02 8.1354421e-01 4.4579946e-02
  1.6339833e-02 1.1145369e-02]]
Max:  0.07597962766885757
Max:  0.8135442137718201
index :  3
[15] (1,)
[15]
[168 170 448 303]
Crop Image:  (133, 280, 3)
(48, 48)
prediction:  [[2.7961415e-01 4.5874098e-05 9.1425218e-02 3.1953424e-02 1.8329239e-01
  2.0582981e-01 2.0783910e-01]]
Max:  0.2796141505241394
index :  0
[15] (1,)
[15]
[162 172 461 302]
Crop Image:  (130, 299, 3)
(48, 48)
prediction:  [[1.6625805e-01 1.2184357e-06 1.4864340e-01 7.2941601e-02 1.2059675e-01
  8.6327761e-02 4.0523127e-01]]
Max:  0.1662580519914627
Max:  0.4052312672138214
index :  6
[15] (1,)
[15]
[169 168 444 305]
Crop Image:  (137, 275, 3)
(48, 48)
prediction:  [[2.2362183e-01 1.0052188e-06 4.6275843e-02 2.9303992e-02 3.5147727e-01
  2.4711886e-02 3.2460818e-01]]
Max:  0.22362183034420013
Max:  0.3514772653579712
index :  4
[15] (1,)
[15]
[164 174 456 300]
Cr